In [1]:
import sys
import os
#dir_path = os.path.dirname(os.path.realpath(__file__))
#sys.path.append(dir_path)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse
import scipy.io
import numpy as np
import deepxde as dde # version 0.11 or higher
from deepxde.backend import tf

Using backend: tensorflow.compat.v1

2023-02-17 16:16:33.253090: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 16:16:33.460368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 16:16:33.460413: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 16:16:34.323852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object 

Instructions for updating:
non-resource variables are not supported in the long term


2023-02-17 16:16:35.933924: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-17 16:16:35.934002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cx3-6-29.cx3.hpc.ic.ac.uk): /proc/driver/nvidia/version does not exist


In [2]:
## PDE Parameters (initialized for 1D PINN)

input = 3
num_hidden_layers = 5
hidden_layer_size = 60
num_domain = 40000
num_boundary = 4000
epochs_main = 150000
output = 2 # network input size 
        
## Training Parameters
num_domain = 20000 # number of training points within the domain
num_boundary = 1000 # number of training boundary condition points on the geometry boundary
num_test = 1000 # number of testing points within the domain
MAX_MODEL_INIT = 16 # maximum number of times allowed to initialize the model
MAX_LOSS = 4 # upper limit to the initialized loss
epochs_init = 15000 # number of epochs for training initial phase
epochs_main = 10000 # number of epochs for main training phase
lr = 0.0005 # learning rate

In [3]:
a = 0.01
b = 0.15
D = 0.1
k = 8
mu_1 = 0.2
mu_2 = 0.3
epsilon = 0.002

## Geometry Parameters
min_x = 0.1
max_x = 10
min_y = 0.1
max_y = 10
min_t = 1
max_t = 50
spacing = 0.1

In [4]:
file_name = "data_2d_corner.mat"
data = scipy.io.loadmat(file_name)
t, x, y, Vsav, Wsav = data["t"], data["x"], data["y"], data["Vsav"], data["Wsav"]
X, T, Y = np.meshgrid(x,t,y)
Y = Y.reshape(-1, 1)

max_t = np.max(t)
max_x = np.max(x)        
X = X.reshape(-1, 1)
T = T.reshape(-1, 1)
V = Vsav.reshape(-1, 1)
W = Wsav.reshape(-1, 1)

In [5]:
observe_x = np.hstack((X, Y, T))

In [6]:
def IC_func(observe_train, v_train):
        
        T_ic = observe_train[:,-1].reshape(-1,1)
        idx_init = np.where(np.isclose(T_ic,1))[0]
        v_init = v_train[idx_init]
        observe_init = observe_train[idx_init]
        return dde.PointSetBC(observe_init,v_init,component=0)

In [7]:
def boundary_func_2d(x, on_boundary):
    return on_boundary and ~(x[0:2]==[min_x,min_y]).all() and  ~(x[0:2]==[min_x,max_y]).all() and ~(x[0:2]==[max_x,min_y]).all()  and  ~(x[0:2]==[max_x,max_y]).all() 


In [8]:
def BC_func(geomtime):
        bc = dde.NeumannBC(geomtime, lambda x:  np.zeros((len(x), 1)), boundary_func_2d, component=0)
        return bc

In [9]:
observe_train, observe_test, v_train, v_test, w_train, w_test = train_test_split(observe_x,V,W,test_size=0.8)  

In [10]:
geom = dde.geometry.Rectangle([min_x,min_y], [max_x,max_y])
timedomain = dde.geometry.TimeDomain(min_t, max_t)
geomtime = dde.geometry.GeometryXTime(geom, timedomain) 

In [11]:
## Define Boundary Conditions
bc = BC_func(geomtime)

## Define Initial Conditions
ic = IC_func(observe_train, v_train)
    
## Model observed data
observe_v = dde.PointSetBC(observe_train, v_train, component=0)
input_data = [bc, ic, observe_v]

In [12]:
net = dde.maps.FNN([input] + [hidden_layer_size] * num_hidden_layers + [output], "tanh", "Glorot uniform")

In [13]:
def pde_2D(x, y):
    print(y.shape)
    print(x.shape)
    
    V, W = y[:, 0:1], y[:, 1:2]
    print(V)
    print(W)
    print(dde.grad.hessian(y, x, component=0, i=0, j=0))
    print(dde.grad.jacobian(y, x, i=0, j=1))
    dv_dt = dde.grad.jacobian(y, x, i=0, j=2)
    dv_dxx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dv_dyy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    dw_dt = dde.grad.jacobian(y, x, i=1, j=2)
        ## Coupled PDE+ODE Equations
    eq_a = dv_dt -  D*(dv_dxx + dv_dyy) + k*V*(V-a)*(V-1) +W*V 
    eq_b = dw_dt -  (epsilon + (mu_1*W)/(mu_2+V))*(-W -k*V*(V-b-1))
    return [eq_a, eq_b]

In [14]:
pde = pde_2D

In [15]:
pde_data = dde.data.TimePDE(geomtime, pde, input_data,
                            num_domain = num_domain, 
                            num_boundary=num_boundary, 
                            anchors=observe_train,
                            num_test=num_test) 
model = dde.Model(pde_data, net)


In [16]:
model.compile("adam", lr=lr)

Compiling model...
Building feed-forward neural network...
'build' took 0.085063 s

(None, 2)
(None, 3)
Tensor("strided_slice:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_1:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_4:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_5:0", shape=(None, 1), dtype=float32)


/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:114: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
2023-02-17 16:16:57.070191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 2)
(None, 3)
Tensor("strided_slice_17:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_18:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_19:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_20:0", shape=(None, 1), dtype=float32)
'compile' took 0.869349 s



In [17]:
def stable_init(model):
    ## Stabalize initialization process by capping the losses
    losshistory, _ = model.train(epochs=1)
    initial_loss = max(losshistory.loss_train[0])
    num_init = 1
    while initial_loss>MAX_LOSS or np.isnan(initial_loss):
        num_init += 1
        model = dde.Model(pde_data, net)
        model.compile("adam", lr=lr)
        losshistory, _ = model.train(epochs=1)
        initial_loss = max(losshistory.loss_train[0])
        if num_init > MAX_MODEL_INIT:
            raise ValueError('Model initialization phase exceeded the allowed limit')
    return 0

In [18]:
out_path = 'test.txt'

In [19]:
def train_3_phase(self, out_path):
    init_weights = [0,0,0,0,1]
        
    ## Initial phase
    model.compile("adam", lr=0.0005, loss_weights=init_weights)
    losshistory, train_state = model.train(epochs=epochs_init, model_save_path = out_path)
    ## Main phase
    model.compile("adam", lr=lr)
    losshistory, train_state = model.train(epochs=epochs_main, model_save_path = out_path)
    ## Final phase
    model.compile("L-BFGS-B")
    losshistory, train_state = model.train(model_save_path = out_path)
    return losshistory, train_state

In [20]:
def train(model, out_path):
        
        ## Stabalize initialization process by capping the losses
        stable_init(model)
      
     
        losshistory, train_state = train_3_phase(model, out_path)
                   
        return model, losshistory, train_state

In [ ]:
model, losshistory, train_state = train(model, out_path)

Initializing variables...


2023-02-17 16:17:05.146770: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


Training model...

Step      Train loss                                            Test loss                                             Test metric
0         [3.25e+00, 1.77e+04, 2.27e-03, 1.80e-01, 3.48e-01]    [2.31e+00, 6.99e+02, 2.27e-03, 1.80e-01, 3.48e-01]    []  
1         [2.35e+01, 6.72e+06, 2.37e-03, 3.26e-01, 6.71e-01]    [2.06e+01, 1.36e+03, 2.37e-03, 3.26e-01, 6.71e-01]    []  

Best model at step 0:
  train loss: 1.77e+04
  test loss: 7.02e+02
  test metric: []

'train' took 6.695508 s

Compiling model...
(None, 2)
(None, 3)
Tensor("strided_slice_30:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_31:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_32:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_33:0", shape=(None, 1), dtype=float32)
(None, 2)
(None, 3)
Tensor("strided_slice_43:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_44:0", shape=(None, 1), dtype=float32)
Tensor("strided_slice_45:0", shape=(None, 1), dtype=float32)
Tensor(

2023-02-17 16:18:03.699591: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_5/bias/Adam_1/Assign' id:2033 op device:{requested: '', assigned: ''} def:{{{node dense_5/bias/Adam_1/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_5/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_5/bias/Adam_1, dense_5/bias/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-02-17 16:18:04.311065: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_5/bias/Adam_17/Assign' id:16595 op device:{requested: '', assigned: ''} def:{{{node dense_5/bias/Adam_17/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_5/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_5/bias/Adam_17, dense_5/bias/Adam_17/Initializer/zeros)}}' was changed by se

Training model...

Step      Train loss                                            Test loss                                             Test metric
1         [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 6.71e-01]    [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 6.71e-01]    []  
1000      [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 2.69e-03]    [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 2.69e-03]    []  


In [ ]:
pred = model.predict(observe_test)
v_pred = pred[:,0:1]
rmse_v = np.sqrt(np.square(v_pred - v_test).mean())
print("V rMSE for test data:", rmse_v)